In [1]:
from naive_bayes import init, preprocesser, test_instances, split_into_train_test

dataset, features, continuous_features, target, possible_features_mrs2, possible_features_mrs3 = init()

dataset = preprocesser(dataset, target, continuous_features)

train_ds, test_ds = split_into_train_test(dataset,0.8)

print('Naive Bayes común')
print('m=1: ',test_instances(train_ds,test_ds,1,False))
print('m=10: ',test_instances(train_ds,test_ds,10,False))
print('m=100: ',test_instances(train_ds,test_ds,100,False))
print('m=1000: ',test_instances(train_ds,test_ds,1000,False))

print('Naive Bayes logarítmico')
print('m=1: ',test_instances(train_ds,test_ds,1,True))
print('m=10: ',test_instances(train_ds,test_ds,10,True))
print('m=100: ',test_instances(train_ds,test_ds,100,True))
print('m=1000: ',test_instances(train_ds,test_ds,1000,True))


Naive Bayes común
m=1:  81.07476635514018
m=10:  81.30841121495327
m=100:  81.07476635514018
m=1000:  77.33644859813083
Naive Bayes logarítmico
m=1:  81.07476635514018
m=10:  81.30841121495327
m=100:  81.07476635514018
m=1000:  77.33644859813083


#### KBins and OneHotEncoding preprocessing ####

In [2]:
from sklearn.model_selection import train_test_split
from naive_bayes import init, preprocesser, test_instances, split_into_train_test
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import OneHotEncoder

dataset, features, continuous_features, target, possible_features_mrs2, possible_features_mrs3 = init()

kbins = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
dataset[continuous_features] = kbins.fit_transform(dataset[continuous_features])

discrete_features = ['trt','race','gender']

X = dataset.drop(target, axis=1)
y = dataset[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

ohe = OneHotEncoder(sparse_output=False)

for feat in discrete_features:
    ohe.fit(dataset[feat].to_numpy().reshape(-1, 1))
    
    # Transform the training and test data using the fitted OneHotEncoder
    # Converts the categorical feature in X_train and X_test to one-hot encoded format
    new_train = ohe.transform(X_train[feat].to_numpy().reshape(-1,1))
    new_test = ohe.transform(X_test[feat].to_numpy().reshape(-1,1))
    
    # Create column names for the new one-hot encoded features
    column_names = [f"{feat}_{cat}" for cat in ohe.categories_[0]]
    
    for i, col_name in enumerate(column_names):
        # Add the new one-hot encoded columns to the X_train and X_test DataFrame
        X_train[col_name] = new_train[:, i]
        X_test[col_name] = new_test[:, i]
    
    X_train.drop(feat, axis=1)
    X_test.drop(feat, axis=1)

train_ds = X_train.join(y_train)
test_ds = X_test.join(y_test)

print('Naive Bayes logarítmico')
print('m=1: ',test_instances(train_ds,test_ds,1,True))
print('m=10: ',test_instances(train_ds,test_ds,10,True))
print('m=100: ',test_instances(train_ds,test_ds,100,True))
print('m=1000: ',test_instances(train_ds,test_ds,1000,True))


Naive Bayes logarítmico
m=1:  83.8785046728972
m=10:  83.41121495327103
m=100:  81.54205607476635
m=1000:  74.29906542056075
